In [1]:
!pip install inverse_optical_flow
!pip install qoi
!pip install efficientnet-pytorch

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for inverse_optical_flow: filename=inverse_optical_flow-0.0.2-cp311-cp311-linux_x86_64.whl size=80856 sha256=47f6d08b06c7e4ee6f5f9db9a443e90a02dc4dc51010394573dd2519a412b9f1
  Stored in directory: /root/.cache/pip/wheels/b1/ff/46/de64bc5cdf61ecf5abbd8bf3f09875f32f56af9ca0bcb89670
Successfully built inverse_optical_flow
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 35.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for qoi: filename=qoi-0.7.0-cp311-cp311-linux_x86_64.whl size=461509 sha256=d9e78d076eb839c0d2b2ab7ad0d74b6bc5f7d9d5676dbe526a603c791c9d0d8a
  Stored in directory: /root/.cache/pip/wheels/b5/39/54/f8024e9295703dbb200aed3c9b57c5a6214612caee5f73872a
Successfully built qoi
  Preparing metadata (setup

In [35]:
import os

import cv2
import numpy as np
import qoi
import torch
import torchvision
import torchvision.models.optical_flow
from torchvision import transforms
from tqdm import tqdm
import ssl



class DivisibleBy8:
    def __call__(self, x: torch.Tensor | np.ndarray) -> torch.Tensor | np.ndarray:
        """ Preprocesses a tensor to be divisible by 8. This is required by the RAFT model. """
        h, w = x.shape[:2]
        h = h - h % 8
        w = w - w % 8
        x = x[:h, :w, ...]
        return x


def frame_to_content(frame: np.ndarray) -> np.ndarray:
    """
    Input: (H, W, C) ndarray, BGR or BGRA
    Output: (H, W, 3) ndarray, RGB
    """
    transform = transforms.Compose([
        DivisibleBy8(),
        torchvision.transforms.Lambda(lambda x: x[..., [2, 1, 0]]),  # BGR to RGB
    ])
    return transform(frame)


def frame_to_tensor(frame: np.ndarray, mean: torch.Tensor, std: torch.Tensor) -> torch.Tensor:
    """
    Input: (H, W, C) ndarray, BGR or BGRA
    Output: (C, H, W) Tensor, normalized
    """
    transform = transforms.Compose([
        DivisibleBy8(),
        torchvision.transforms.Lambda(lambda x: x[..., [2, 1, 0]]),  # BGR to RGB
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
    return transform(frame)


def main():
    ssl._create_default_https_context = ssl._create_unverified_context
    video_filepath = '/content/drive/MyDrive/video_style_transfer/tracked_output_blur.mp4'
    root_path = "/content/drive/MyDrive/video_style_transfer"

    # logger = logging.getLogger(__name__)
    # logger.setLevel(logging.DEBUG)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = torchvision.models.optical_flow.raft_large(weights=torchvision.models.optical_flow.Raft_Large_Weights.C_T_V2).to(device)
    # model = torchvision.models.optical_flow.raft_small(weights=torchvision.models.optical_flow.Raft_Small_Weights.C_T_V2).to(device)
    model.eval()

    raft_mean = torch.tensor((0.5, 0.5, 0.5)).to(device)
    raft_std = torch.tensor((0.5, 0.5, 0.5)).to(device)

    vidcap = cv2.VideoCapture(video_filepath)
    frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    content_prev = None

    for frame_idx in tqdm(range(frame_count), desc="Splitting video and computing optical flow"):
        frame_path = f"{root_path}/frame/{frame_idx}"
        flow_path = f"{root_path}/flow"

        os.makedirs(frame_path, exist_ok=True)
        os.makedirs(flow_path, exist_ok=True)

        content_filepath = f"{frame_path}/content.qoi"
        # forward_flow_filepath = f"{flow_path}/flow_{frame_idx-1}_to_{frame_idx}.npy"
        # backward_flow_filepath = f"{flow_path}/flow_{frame_idx}_to_{frame_idx-1}.npy"
        forward_flow_filepath = f"{flow_path}/flow_{frame_idx-1}_to_{frame_idx}.npz"
        backward_flow_filepath = f"{flow_path}/flow_{frame_idx}_to_{frame_idx-1}.npz"

        success, frame_cur = vidcap.read()
        if not success:
            raise Exception(f"Failed to read frame {frame_idx} from video of {frame_count} frames")

        content = frame_to_content(frame_cur).copy()
        _ = qoi.write(content_filepath, content)

        content_cur = frame_to_tensor(frame_cur, raft_mean, raft_std).to(device)

        if content_prev is not None:
            if not os.path.exists(forward_flow_filepath):
                flow: list[torch.Tensor] = model(content_prev[None], content_cur[None])
                # np.save(forward_flow_filepath, flow[0][0].cpu().detach().numpy())
                np.savez_compressed(forward_flow_filepath, flow[0][0].cpu().detach().numpy())

            # if not os.path.exists(backward_flow_filepath):
            #     flow: list[torch.Tensor] = model(content_cur[None], content_prev[None])
            #     np.save(backward_flow_filepath, flow[0][0].cpu().detach().numpy())
            #     np.savez_compressed(backward_flow_filepath, flow[0][0].cpu().detach().numpy())

        content_prev = content_cur


if __name__ == '__main__':
    main()


Splitting video and computing optical flow: 100%|██████████| 102/102 [00:55<00:00,  1.85it/s]


In [36]:
import os

# 변경하려는 디렉토리 경로
directory_path = "/content/drive/MyDrive/video_style_transfer/flow"

# 디렉토리 내 모든 파일의 확장자 변경
for filename in os.listdir(directory_path):
    if filename.endswith(".npz"):  # .npz 확장자를 가진 파일 찾기
        old_file = os.path.join(directory_path, filename)
        new_file = os.path.join(directory_path, filename.replace(".npz", ".npy"))
        os.rename(old_file, new_file)  # 파일 이름 변경
        print(f"Renamed: {old_file} -> {new_file}")

print("모든 파일의 확장자가 변경되었습니다!")

Renamed: /content/drive/MyDrive/video_style_transfer/flow/flow_0_to_1.npz -> /content/drive/MyDrive/video_style_transfer/flow/flow_0_to_1.npy
Renamed: /content/drive/MyDrive/video_style_transfer/flow/flow_1_to_2.npz -> /content/drive/MyDrive/video_style_transfer/flow/flow_1_to_2.npy
Renamed: /content/drive/MyDrive/video_style_transfer/flow/flow_2_to_3.npz -> /content/drive/MyDrive/video_style_transfer/flow/flow_2_to_3.npy
Renamed: /content/drive/MyDrive/video_style_transfer/flow/flow_3_to_4.npz -> /content/drive/MyDrive/video_style_transfer/flow/flow_3_to_4.npy
Renamed: /content/drive/MyDrive/video_style_transfer/flow/flow_4_to_5.npz -> /content/drive/MyDrive/video_style_transfer/flow/flow_4_to_5.npy
Renamed: /content/drive/MyDrive/video_style_transfer/flow/flow_5_to_6.npz -> /content/drive/MyDrive/video_style_transfer/flow/flow_5_to_6.npy
Renamed: /content/drive/MyDrive/video_style_transfer/flow/flow_6_to_7.npz -> /content/drive/MyDrive/video_style_transfer/flow/flow_6_to_7.npy
Rename

In [38]:
import math
import os
from contextlib import contextmanager
from functools import partial
from pathlib import Path
from typing import Any, Iterable, Mapping, Callable

import inverse_optical_flow
import numpy as np
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image
from efficientnet_pytorch import EfficientNet
from scipy.ndimage import map_coordinates
from tqdm import tqdm


def load_image(image_path) -> Image.Image:
    image = Image.open(image_path)
    return image


def image_to_tensor(image: Image.Image | np.ndarray, mean: torch.Tensor, std: torch.Tensor) -> torch.Tensor:
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
    return transform(image)


def tensor_to_image(tensor: torch.Tensor, mean: torch.Tensor, std: torch.Tensor) -> Image.Image:
    transform = transforms.Compose([
        transforms.Normalize((-mean / std).tolist(), (1.0 / std).tolist()),
        transforms.ToPILImage()
    ])
    return transform(tensor)


def flatten_values(a: Iterable | Mapping | Any):
    if isinstance(a, (dict, )):
        for x in a.values():
            yield from flatten_values(x)
    elif isinstance(a, (list, tuple, set)):
        for x in a:
            yield from flatten_values(x)
    else:
        yield a


def total_variation2d(x: torch.Tensor):
    return torch.sum(torch.abs(x[:, :, :-1] - x[:, :, 1:])) + torch.sum(torch.abs(x[:, :-1, :] - x[:, 1:, :]))


@contextmanager
def register_hooks(
        model: torch.nn.Module,
        hook: Callable,
        **kwargs
):
    handles = []
    try:
        for name, module in model.named_modules():
            if isinstance(module, torch.nn.Conv2d):
                hook: Callable = partial(hook, name=name, **kwargs)
                handle = module.register_forward_hook(hook)
                handles.append(handle)
        yield handles
    finally:
        for handle in handles:
            handle.remove()


def stat_recorder_hook(
        module: torch.nn.Module,
        input: torch.Tensor,
        output: torch.Tensor,
        name: str,
        *,
        eps: float | torch.Tensor = 1e-6,
        storage: dict[str, dict[str, torch.Tensor]]
):
    mean = output.mean(dim=[0, 2, 3])
    std = output.std(dim=[0, 2, 3], unbiased=False)
    # skewness = ((output - mean[None, :, None, None]) ** 3).mean(dim=[0, 2, 3]) / (std ** 3 + eps)
    # kurtosis = ((output - mean[None, :, None, None]) ** 4).mean(dim=[0, 2, 3]) / (std ** 4 + eps)
    # assert torch.isfinite(torch.cat([mean, std, skewness, kurtosis])).all()
    storage[name] = {
        "mean": mean,
        "std": std,
        # "skewness": skewness,
        # "kurtosis": kurtosis,
    }


def get_stats(model: torch.nn.Module, image: torch.Tensor) -> dict[str, dict[str, torch.Tensor]]:
    stats = {}
    with register_hooks(model, stat_recorder_hook, storage=stats):
        _ = model(image[None])
    return stats


def alpha_composite(im1, im2, opacity1=1.0, opacity2=1.0):
    """
    Input: (4, H, W) ndarray, RGBA in 0-255
    Output: (4, H, W) ndarray, RGBA in 0-255
    """
    # Validate the opacity values
    if not 0 <= opacity1 <= 1 or not 0 <= opacity2 <= 1:
        raise ValueError('Opacity must be between 0 and 1')

    # Assuming the last channel is the alpha channel
    # Scale the alpha channels by the provided opacity values
    im1[3, :, :] = im1[3, :, :] * opacity1
    im2[3, :, :] = im2[3, :, :] * opacity2

    # Normalize the alpha channels to be between 0 and 1
    im1_alpha = im1[3, :, :] / 255.0
    im2_alpha = im2[3, :, :] / 255.0

    # Compute the composite alpha channel
    composite_alpha = im1_alpha + im2_alpha * (1 - im1_alpha)

    # Handle case where composite_alpha is 0 to avoid divide by zero error
    mask = composite_alpha > 0
    composite_alpha = np.where(mask, composite_alpha, 1)

    # Compute the composite image
    composite_image = np.empty_like(im1)
    for channel in range(3):  # Assuming the first 3 channels are RGB
        composite_image[channel, :, :] = (
            im1[channel, :, :] * im1_alpha
            + im2[channel, :, :] * im2_alpha * (1 - im1_alpha)
        ) / composite_alpha

    # Add the composite alpha channel to the image
    composite_image[3, :, :] = composite_alpha * 255

    return composite_image.astype(np.uint8)


def warp(image: np.ndarray, backward_flow: np.ndarray, order=3) -> np.ndarray:
    channels, height, width = image.shape
    index_grid = np.mgrid[0:height, 0:width].astype(float)
    # Widely, first channel is horizontal x-axis flow, the second channel is vertical y-axis flow.
    print(backward_flow.shape)

    # backward_flow 리사이즈
    #height, width = 536, 960  # 비디오 프레임의 해상도
    backward_flow = cv2.resize(backward_flow.transpose(1, 2, 0), (width, height)).transpose(2, 0, 1)
    coordinates = index_grid + backward_flow[::-1]
    remapped = np.empty(image.shape, dtype=image.dtype)
    for i in range(channels):
        remapped[i] = map_coordinates(image[i], coordinates, order=order, mode='constant', cval=0)
    return remapped


class PlateuPruner:
    def __init__(self, patience: int = 5, min_delta: float = 0.0, target='minimize'):
        """
        Initializes the EarlyStopping instance.
        :param patience: Number of epochs with no improvement after which training will be stopped.
        :param min_delta: Minimum change in the monitored quantity to qualify as an improvement.
        :param target: 'minimize' for minimizing a metric (like loss), 'maximize' for maximizing a metric (like accuracy).
        """
        self.patience = patience
        self.min_delta = min_delta
        self.target = target
        self.counter = 0
        self.best_score = math.inf if target == 'minimize' else -math.inf
        self.early_stop = False
        self.is_best = True

    def report(self, score):
        """
        Reports the latest metric value and checks if it's an improvement.
        :param metric_value: The latest metric value (e.g., validation loss or accuracy).
        """
        if self.target == 'minimize':
            self.is_best = score < self.best_score
            is_improvement = score < self.best_score - self.min_delta
        else:
            self.is_best = score > self.best_score
            is_improvement = score > self.best_score + self.min_delta

        if is_improvement:
            self.best_score = score
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        return self.is_best, self.early_stop


def main():
    root_path = "'/content/drive/MyDrive/video_style_transfer"
    frames_path = '/content/drive/MyDrive/video_style_transfer/frame'
    flow_path = '/content/drive/MyDrive/video_style_transfer/flow'

    style_filepath = "/content/drive/MyDrive/video_style_transfer/mosaic.jpg"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # model = torchvision.models.vgg16(pretrained=True).features
    # model = torchvision.models.resnet18(pretrained=True)
    model = EfficientNet.from_pretrained('efficientnet-b0')
    # model = EfficientNet.from_pretrained('efficientnet-b4')

    # Disable grad
    for param in model.parameters():
        param.requires_grad_(False)
    # Disable running stats
    for module in model.modules():
        if isinstance(module, torch.nn.modules.batchnorm._BatchNorm):
            module.track_running_stats = False
    # Unset "inplace"
    for module in model.modules():
        if hasattr(module, "inplace"):
            module.inplace = False

    model = model.to(device)
    model.eval()
    # print(f"model: {model}")

    named_layers = [name for name, module in model.named_modules() if isinstance(module, torch.nn.Conv2d)]
    style_content_weights_per_layer = {
        name:
            (1, 1)
        for i, name in enumerate(named_layers)
    }
    print(f"style_content_weights_per_layer: {style_content_weights_per_layer}")

    style_weight = 1e+1
    content_weight = 1e+0
    temporal_weight = 1e+2
    total_variation_weight = 0

    mean = torch.tensor((0.485, 0.456, 0.406)).to(device)
    std = torch.tensor((0.229, 0.224, 0.225)).to(device)

    # Clamping range for normalized image
    min_vals = (0 - mean) / std
    max_vals = (1 - mean) / std

    style_image = load_image(style_filepath)
    style_image = image_to_tensor(style_image, mean, std).to(device)
    style_stats = get_stats(model, style_image)
    # print(f"style_stats.keys(): {style_stats.keys()}")
    assert torch.isfinite(torch.cat(list(flatten_values(style_stats)))).all()

    frame_indices = sorted([int(x.name) for x in os.scandir(frames_path) if x.is_dir()])
    print(f"frames: {frame_indices}")

    styled = None
    styled_prev_warped = None

    for frame_i in frame_indices:
        frame_path = f"{frames_path}/{frame_i}"
        for filepath in Path(frame_path).glob("styled_*.png"):
            os.remove(filepath)

    for frame_i in tqdm(frame_indices, desc="Styling"):
        frame_path = f"{frames_path}/{frame_i}"

        content_filepath = f"{frame_path}/content.qoi"
        styled_filepath = f"{frame_path}/styled.pt"
        forward_flow_filepath = f"{flow_path}/flow_{frame_i-1}_to_{frame_i}.npy"
        backward_flow_filepath = f"{flow_path}/flow_{frame_i}_to_{frame_i-1}.npy"

        content = load_image(content_filepath)
        content = image_to_tensor(content, mean, std).to(device)
        tensor_to_image(content, mean, std).save(f"{frame_path}/0_content.png")

        if styled is None:
            styled = content.clone().to(device)
            disocclusion_mask = torch.ones_like(styled, device=device)
        else:
            forward_flow = np.load(forward_flow_filepath)

            with torch.no_grad():
                styled.data = styled.data.clamp_(min_vals[:, None, None], max_vals[:, None, None])
            np_styled = tensor_to_image(styled, mean, std).convert('RGBA')
            np_styled = np.array(np_styled).transpose(2, 0, 1)

            np_content = tensor_to_image(content, mean, std).convert('RGBA')
            np_content = np.array(np_content).transpose(2, 0, 1)

            # 적절한 키로 데이터 가져오기
            forward_flow = data['arr_0']  # 키 확인 후 수정

            # Numpy 배열로 변환 (필요 시 float32로 변환)
            forward_flow = np.array(forward_flow, dtype=np.float32)  # Numpy 배열로 변환

            # max_method 호출
            forward_flow_inv, disocclusion_mask = inverse_optical_flow.max_method(forward_flow)

            #forward_flow_inv, disocclusion_mask = inverse_optical_flow.max_method(forward_flow)
            np_styled = warp(np_styled, forward_flow_inv, order=3)


            #disocclusion_mask와  np_styled간의 shape 맞추기
            # disocclusion_mask를 비디오 프레임 크기에 맞게 리사이즈
            height, width = np_styled.shape[1:]  # 비디오 프레임 크기 가져오기

            print(f"height, width {height, width}")
            disocclusion_mask = cv2.resize(disocclusion_mask, (width, height))

            print(f"resized height, width {disocclusion_mask.shape}")

            np_styled[3, :, :] = np_styled[3, :, :] * (1 - disocclusion_mask)
            np_styled = alpha_composite(np_styled, np_content)
            np_styled = np_styled[:3].transpose(1, 2, 0)
            styled_prev_warped = image_to_tensor(np_styled, mean, std).to(device)

            styled = content.clone().to(device)
            disocclusion_mask = torch.from_numpy(disocclusion_mask).to(device).to(torch.float32)

        styled.requires_grad_(True)

        def get_opimization_strategy():
            # optimizer = torch.optim.LBFGS([styled], lr=1, max_iter=40)
            optimizer = torch.optim.Adam([styled], lr=0.2)
            scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda epoch: 1)
            return optimizer, scheduler

        optimizer, scheduler = get_opimization_strategy()

        epochs = 1 if isinstance(optimizer, torch.optim.LBFGS) else 40

        content_stats = get_stats(model, content)
        assert torch.isfinite(torch.cat(list(flatten_values(content_stats)))).all()

        pruner = PlateuPruner(patience=20, min_delta=0.01, target="minimize")
        best_styled = styled.clone().to(device)

        for epoch in range(epochs):
            iteration = 0

            def closure() -> float:
                nonlocal iteration
                print(f"closure(): frame_i: {frame_i}, epoch: {epoch}, iteration: {iteration}")
                optimizer.zero_grad(set_to_none=True)

                with torch.no_grad():
                    styled.data = styled.data.clamp_(min_vals[:, None, None], max_vals[:, None, None])
                tensor_to_image(styled, mean, std).save(f"{frame_path}/styled_{epoch}_{iteration}.png")
                styled_stats = get_stats(model, styled)

                loss = torch.zeros(1, device=device)
                for name, (style_w, content_w) in style_content_weights_per_layer.items():
                    # If requested layer weight not found in style or content stats, skip it
                    if name not in styled_stats or name not in style_stats or name not in content_stats:
                        continue
                    loss += style_weight * style_w * torch.nn.functional.mse_loss(
                        torch.cat(list(flatten_values(styled_stats[name]))),
                        torch.cat(list(flatten_values(style_stats[name]))))
                    loss += content_weight * content_w * torch.nn.functional.mse_loss(
                        torch.cat(list(flatten_values(styled_stats[name]))),
                        torch.cat(list(flatten_values(content_stats[name]))))
                loss += total_variation_weight * total_variation2d(styled)
                if styled_prev_warped is not None:
                    loss += temporal_weight * torch.nn.functional.mse_loss(styled * (1 - disocclusion_mask),
                                                                           styled_prev_warped * (1 - disocclusion_mask))
                assert not torch.isnan(loss).any()

                print(f"loss: {loss.item()}, lr: {optimizer.param_groups[0]['lr']}")
                iteration += 1
                loss.backward()
                return loss.item()

            closure_loss = optimizer.step(closure)
            if isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                scheduler.step(closure_loss)
            else:
                scheduler.step()

            is_best, should_prune = pruner.report(closure_loss)
            if is_best:
                print(f"New best at frame {frame_i}, epoch {epoch}, iteration {iteration}")
                best_styled = styled.clone().to(device)
            if should_prune:
                print(f"Early stopping at frame {frame_i}, epoch {epoch}, iteration {iteration}")
                break

        styled = best_styled
        torch.save(styled, styled_filepath)


if __name__ == '__main__':
    main()

Loaded pretrained weights for efficientnet-b0
style_content_weights_per_layer: {'_conv_stem': (1, 1), '_blocks.0._depthwise_conv': (1, 1), '_blocks.0._se_reduce': (1, 1), '_blocks.0._se_expand': (1, 1), '_blocks.0._project_conv': (1, 1), '_blocks.1._expand_conv': (1, 1), '_blocks.1._depthwise_conv': (1, 1), '_blocks.1._se_reduce': (1, 1), '_blocks.1._se_expand': (1, 1), '_blocks.1._project_conv': (1, 1), '_blocks.2._expand_conv': (1, 1), '_blocks.2._depthwise_conv': (1, 1), '_blocks.2._se_reduce': (1, 1), '_blocks.2._se_expand': (1, 1), '_blocks.2._project_conv': (1, 1), '_blocks.3._expand_conv': (1, 1), '_blocks.3._depthwise_conv': (1, 1), '_blocks.3._se_reduce': (1, 1), '_blocks.3._se_expand': (1, 1), '_blocks.3._project_conv': (1, 1), '_blocks.4._expand_conv': (1, 1), '_blocks.4._depthwise_conv': (1, 1), '_blocks.4._se_reduce': (1, 1), '_blocks.4._se_expand': (1, 1), '_blocks.4._project_conv': (1, 1), '_blocks.5._expand_conv': (1, 1), '_blocks.5._depthwise_conv': (1, 1), '_blocks.5.

Styling:   0%|          | 0/102 [00:00<?, ?it/s]

closure(): frame_i: 0, epoch: 0, iteration: 0
loss: 4845.82373046875, lr: 0.2
New best at frame 0, epoch 0, iteration 1
closure(): frame_i: 0, epoch: 1, iteration: 0
loss: 3144.03125, lr: 0.2
New best at frame 0, epoch 1, iteration 1
closure(): frame_i: 0, epoch: 2, iteration: 0
loss: 2202.09326171875, lr: 0.2
New best at frame 0, epoch 2, iteration 1
closure(): frame_i: 0, epoch: 3, iteration: 0
loss: 1740.09765625, lr: 0.2
New best at frame 0, epoch 3, iteration 1
closure(): frame_i: 0, epoch: 4, iteration: 0
loss: 1509.3851318359375, lr: 0.2
New best at frame 0, epoch 4, iteration 1
closure(): frame_i: 0, epoch: 5, iteration: 0
loss: 1405.638427734375, lr: 0.2
New best at frame 0, epoch 5, iteration 1
closure(): frame_i: 0, epoch: 6, iteration: 0
loss: 1351.808349609375, lr: 0.2
New best at frame 0, epoch 6, iteration 1
closure(): frame_i: 0, epoch: 7, iteration: 0
loss: 1319.530029296875, lr: 0.2
New best at frame 0, epoch 7, iteration 1
closure(): frame_i: 0, epoch: 8, iteration: 

Styling:   1%|          | 1/102 [00:11<18:52, 11.22s/it]

loss: 581.5300903320312, lr: 0.2
New best at frame 0, epoch 39, iteration 1
(2, 432, 1024)
height, width (536, 960)
resized height, width (536, 960)
closure(): frame_i: 1, epoch: 0, iteration: 0
loss: 4623.0869140625, lr: 0.2
New best at frame 1, epoch 0, iteration 1
closure(): frame_i: 1, epoch: 1, iteration: 0
loss: 3065.18896484375, lr: 0.2
New best at frame 1, epoch 1, iteration 1
closure(): frame_i: 1, epoch: 2, iteration: 0
loss: 2182.4736328125, lr: 0.2
New best at frame 1, epoch 2, iteration 1
closure(): frame_i: 1, epoch: 3, iteration: 0
loss: 1757.5533447265625, lr: 0.2
New best at frame 1, epoch 3, iteration 1
closure(): frame_i: 1, epoch: 4, iteration: 0
loss: 1539.7838134765625, lr: 0.2
New best at frame 1, epoch 4, iteration 1
closure(): frame_i: 1, epoch: 5, iteration: 0
loss: 1450.5836181640625, lr: 0.2
New best at frame 1, epoch 5, iteration 1
closure(): frame_i: 1, epoch: 6, iteration: 0
loss: 1401.80078125, lr: 0.2
New best at frame 1, epoch 6, iteration 1
closure():

...

loss: 590.2069702148438, lr: 0.2
New best at frame 99, epoch 39, iteration 1
(2, 432, 1024)
height, width (536, 960)
resized height, width (536, 960)
closure(): frame_i: 100, epoch: 0, iteration: 0
loss: 4334.6162109375, lr: 0.2
New best at frame 100, epoch 0, iteration 1
closure(): frame_i: 100, epoch: 1, iteration: 0
loss: 2864.91650390625, lr: 0.2
New best at frame 100, epoch 1, iteration 1
closure(): frame_i: 100, epoch: 2, iteration: 0
loss: 2065.980712890625, lr: 0.2
New best at frame 100, epoch 2, iteration 1
closure(): frame_i: 100, epoch: 3, iteration: 0
loss: 1691.0157470703125, lr: 0.2
New best at frame 100, epoch 3, iteration 1
closure(): frame_i: 100, epoch: 4, iteration: 0
loss: 1509.25244140625, lr: 0.2
New best at frame 100, epoch 4, iteration 1
closure(): frame_i: 100, epoch: 5, iteration: 0
loss: 1424.4642333984375, lr: 0.2
New best at frame 100, epoch 5, iteration 1
closure(): frame_i: 100, epoch: 6, iteration: 0
loss: 1377.412841796875, lr: 0.2
New best at frame 100

Styling:  99%|█████████▉| 101/102 [20:26<00:12, 12.28s/it]

loss: 587.7774047851562, lr: 0.2
New best at frame 100, epoch 39, iteration 1
(2, 432, 1024)
height, width (536, 960)
resized height, width (536, 960)
closure(): frame_i: 101, epoch: 0, iteration: 0
loss: 4336.47607421875, lr: 0.2
New best at frame 101, epoch 0, iteration 1
closure(): frame_i: 101, epoch: 1, iteration: 0
loss: 2866.70263671875, lr: 0.2
New best at frame 101, epoch 1, iteration 1
closure(): frame_i: 101, epoch: 2, iteration: 0
loss: 2065.18359375, lr: 0.2
New best at frame 101, epoch 2, iteration 1
closure(): frame_i: 101, epoch: 3, iteration: 0
loss: 1690.86865234375, lr: 0.2
New best at frame 101, epoch 3, iteration 1
closure(): frame_i: 101, epoch: 4, iteration: 0
loss: 1509.280029296875, lr: 0.2
New best at frame 101, epoch 4, iteration 1
closure(): frame_i: 101, epoch: 5, iteration: 0
loss: 1424.6005859375, lr: 0.2
New best at frame 101, epoch 5, iteration 1
closure(): frame_i: 101, epoch: 6, iteration: 0
loss: 1377.5767822265625, lr: 0.2
New best at frame 101, epo

Styling: 100%|██████████| 102/102 [20:38<00:00, 12.14s/it]

loss: 588.277099609375, lr: 0.2
New best at frame 101, epoch 39, iteration 1


NameError: name 'backward_flow' is not defined

In [39]:
import os

import cv2
import numpy as np
import torch
from torchvision import transforms
from tqdm import tqdm


def tensor_to_frame(tensor: torch.Tensor, means: torch.Tensor, stds: torch.Tensor) -> np.ndarray:
    """
    Input: (C, H, W) Tensor, normalized, RGB
    Output: (H, W, C) ndarray, BGR
    """
    transform = transforms.Compose([
        transforms.Normalize((-means / stds).tolist(), (1.0 / stds).tolist()),
        transforms.Lambda(lambda x: x[[2, 1, 0], ...]),  # RGB to BGR
        transforms.ToPILImage(),
        transforms.Lambda(lambda x: np.array(x))
    ])
    return transform(tensor)


def main():
    video_filepath = "/content/drive/MyDrive/video_style_transfer/tracked_output_blur.mp4"
    root_path = "/content/drive/MyDrive/video_style_transfer"
    frames_path = "/content/drive/MyDrive/video_style_transfer/frame"
    output_filepath, fourcc = "/content/drive/MyDrive/video_style_transfer/output/tracking.mp4/tracking_transfer_v4.mp4", "mp4v"
    # output_filepath, fourcc = "output/sintel.mp4/out_vp90.webm", "VP90"
    # output_filepath, fourcc = "output/sintel.mp4/out_ffv1.avi", "FFV1"
    # output_filepath, fourcc = "output/sintel.mp4/out_xvid.avi", "xvid"
    # output_filepath, fourcc = "output/sintel.mp4/out_mjpg.avi", "MJPG"

    os.makedirs(os.path.dirname(output_filepath), exist_ok=True)

    mean = torch.tensor((0.485, 0.456, 0.406))
    std = torch.tensor((0.229, 0.224, 0.225))

    # Clamping range for normalized image
    min_vals = (0 - mean) / std
    max_vals = (1 - mean) / std

    vidcap = cv2.VideoCapture(video_filepath)
    frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = sorted([int(x.name) for x in os.scandir(frames_path) if x.is_dir()])
    if len(frame_indices) != frame_count:
        print("WARNING: Number of frames in video does not match number of frames in root directory")
    fourcc = cv2.VideoWriter_fourcc(*fourcc)

    out = None

    for frame_i in tqdm(frame_indices, desc="Joining video"):
        frame_path = f"{frames_path}/{frame_i}"

        styled_filepath = f"{frame_path}/styled.pt"
        styled = torch.load(styled_filepath)
        # styled.data의 디바이스 확인
        device = styled.data.device

        # min_vals와 max_vals를 동일한 디바이스로 이동
        min_vals = min_vals.to(device)
        max_vals = max_vals.to(device)

        styled.data = styled.data.clamp_(min_vals[:, None, None], max_vals[:, None, None])
        styled = tensor_to_frame(styled, mean, std)
        h, w, c = styled.shape
        if c == 4:
            styled = styled[..., :3]
        if out is None:
            frame_shape = w, h
            out = cv2.VideoWriter(output_filepath, fourcc, vidcap.get(cv2.CAP_PROP_FPS), frame_shape)
        out.write(styled)

    if out is not None:
        out.release()


if __name__ == '__main__':
    main()


Joining video:   0%|          | 0/102 [00:00<?, ?it/s]<ipython-input-39-ba44c4efc96f>:56: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  styled = torch.load(styled_filepath)
